In [1]:
class solution:
    def __init__(self):
        self.best = 0
        self.bestIndividual=[]
        self.convergence = []
        self.optimizer=""
        self.objfname=""
        self.startTime=0
        self.endTime=0
        self.executionTime=0
        self.lb=0
        self.ub=0
        self.dim=0
        self.popnum=0
        self.maxiers=0
        self.no_of_iterations=0
        self.positions=[]

In [2]:
def simplebounds(s,Lb,Ub):
    #% Apply the lower bound
    ns_tmp=np.array(s)
    I=ns_tmp<Lb
    ns_tmp[I]=Lb
  
    #% Apply the upper bounds 
    J=ns_tmp>Ub
    ns_tmp[J]=Ub
    #% Update this new move 
    return ns_tmp

In [86]:
# -*- coding: utf-8 -*-
"""
Created on Mon May 16 00:27:50 2016
@author: Hossam Faris
"""

import random
import numpy
import math
#from solution import solution
import time


    

def GWO(objf,lb,ub,dim,SearchAgents_no,Max_iter):
    
    
    #Max_iter=1000
    #lb=-100
    #ub=100
    #dim=30  
    #SearchAgents_no=
    ggnn=0
    levy=np.zeros(SearchAgents_no)
    
    # initialize alpha, beta, and delta_pos
    Alpha_pos=numpy.zeros(dim)
    Alpha_score=float("inf")
    
    Beta_pos=numpy.zeros(dim)
    Beta_score=float("inf")
    
    Delta_pos=numpy.zeros(dim)
    Delta_score=float("inf")
    
    #Initialize the positions of search agents
    Positions=numpy.random.uniform(0,1,(SearchAgents_no,dim)) *(ub-lb)+lb
    positions = numpy.random.choice([0, 1], size=(SearchAgents_no,dim))
    
    Convergence_curve=numpy.zeros(Max_iter)
    sol=solution()

     # Loop counter
    #print("GWO is optimizing  \""+objf.__name__+"\"")    
    
    timerStart=time.time() 
    sol.startTime=time.strftime("%Y-%m-%d-%H-%M-%S")
    # Main loop
    for l in range(0,Max_iter):
        for i in range(0,SearchAgents_no):
            
            # Return back the search agents that go beyond the boundaries of the search space
            Positions[i,:]=numpy.clip(Positions[i,:], lb, ub)

            # Calculate objective function for each search agent
            fitness=objf(Positions[i,:])
            
            # Update Alpha, Beta, and Delta
            if fitness<Alpha_score :
                Alpha_score=fitness; # Update alpha
                Alpha_pos=Positions[i,:].copy()
            
            
            if (fitness>Alpha_score and fitness<Beta_score ):
                Beta_score=fitness  # Update beta
                Beta_pos=Positions[i,:].copy()
            
            
            if (fitness>Alpha_score and fitness>Beta_score and fitness<Delta_score): 
                Delta_score=fitness # Update delta
                Delta_pos=Positions[i,:].copy()
            
        
        
        z=numpy.random.uniform()
        z=4*z*(1-z)
        
        a=2-l*((2)/Max_iter)+z; # a decreases linearly fron 2 to 0
        
        beta=0.5
        sigma=(math.gamma(1+beta)*math.sin(math.pi*beta/2)/(math.gamma((1+beta)/2)*beta*2**((beta-1)/2)))**(1/beta);
        
        # Update the Position of search agents including omegas
        for i in range(0,SearchAgents_no):
            for j in range (0,dim):     
                           
                r1=random.random() # r1 is a random number in [0,1]
                r2=random.random() # r2 is a random number in [0,1]
                
                A1=2*a*r1-a; # Equation (3.3)
                C1=2*r2; # Equation (3.4)
                
                D_alpha=abs(C1*Alpha_pos[j]-Positions[i,j]); # Equation (3.5)-part 1
                X1=Alpha_pos[j]-A1*D_alpha; # Equation (3.6)-part 1
                #else:
                 #   X1=random.random()
                #X1=numpy.random.uniform(0,1,(SearchAgents_no,dim)) *(ub-lb)+lb
                           
                r1=random.random()
                r2=random.random()
                
                A2=2*a*r1-a; # Equation (3.3)
                C2=2*r2; # Equation (3.4)
                
                D_beta=abs(C2*Beta_pos[j]-Positions[i,j]); # Equation (3.5)-part 2
                #if j%2==1:
                X2=Beta_pos[j]-A2*D_beta; # Equation (3.6)-part 2
                #else:
                #    X2=random.random()
                
                r1=random.random()
                r2=random.random() 
                
                A3=2*a*r1-a; # Equation (3.3)
                C3=2*r2; # Equation (3.4)
                
                D_delta=abs(C3*Delta_pos[j]-Positions[i,j]); # Equation (3.5)-part 3
                X3=Delta_pos[j]-A3*D_delta; # Equation (3.5)-part 3             
                
                Positions[i,j]=(X1+X2+X3)/3  # Equation (3.7)
                #rr=levy(,j)
                
            
            #ggnn=alpha(i,Max_iter)
            s=Positions[i,:]
            
            u=numpy.random.uniform(size=len(s))*sigma
            v=numpy.random.uniform(size=len(s))
            step=u*abs(v)**(1/beta)
            stepsize=0.1*step*(s-Alpha_pos) #s-alpha_pos
            s=s+stepsize*numpy.random.uniform(size=len(s))
            Positions[i]=simplebounds(s,lb,ub)
            rand=random.random()
            if(sigmoid(Positions[i-1,j])>=rand):
                positions[i,j]=1

            else:
                positions[i,j]=0
        Convergence_curve[l]=Alpha_score
        """if Convergence_curve[l]==Convergence_curve[l-1]==Convergence_curve[l-2]==Convergence_curve[l-3]==Convergence_curve[l-4]==Convergence_curve[l-5]==Convergence_curve[l-6]:
            print(Convergence_curve[l],Convergence_curve[l-6])
            break"""

        """if (l%1==0):
               print(['At iteration '+ str(l)+ ' the best fitness is '+ str(Alpha_score)]);"""
    
    timerEnd=time.time()  
    sol.endTime=time.strftime("%Y-%m-%d-%H-%M-%S")
    sol.executionTime=timerEnd-timerStart
    sol.convergence=Convergence_curve
    sol.optimizer="GWO"
    sol.objfname=objf.__name__
    sol.no_of_iterations=l
    sol.positions=positions
    
    
    return sol
  
  

In [116]:
# -*- coding: utf-8 -*-
"""
Created on Mon May 16 00:27:50 2016
@author: Hossam Faris
"""

import random
import numpy
import math
#from solution import solution
import time


    

def GWO2(objf,lb,ub,dim,SearchAgents_no,Max_iter):
    
    
    #Max_iter=1000
    #lb=-100
    #ub=100
    #dim=30  
    #SearchAgents_no=
    ggnn=0
    levy=np.zeros(SearchAgents_no)
    
    # initialize alpha, beta, and delta_pos
    Alpha_pos=numpy.zeros(dim)
    Alpha_score=float("inf")
    
    Beta_pos=numpy.zeros(dim)
    Beta_score=float("inf")
    
    Delta_pos=numpy.zeros(dim)
    Delta_score=float("inf")
    
    #Initialize the positions of search agents
    Positions=numpy.random.uniform(0,1,(SearchAgents_no,dim)) *(ub-lb)+lb
    positions = numpy.random.choice([0, 1], size=(SearchAgents_no,dim))
    
    Convergence_curve=numpy.zeros(Max_iter)
    sol=solution()

     # Loop counter
    #print("GWO is optimizing  \""+objf.__name__+"\"")    
    
    timerStart=time.time() 
    sol.startTime=time.strftime("%Y-%m-%d-%H-%M-%S")
    # Main loop
    for l in range(0,Max_iter):
        for i in range(0,SearchAgents_no):
            
            # Return back the search agents that go beyond the boundaries of the search space
            Positions[i,:]=numpy.clip(Positions[i,:], lb, ub)

            # Calculate objective function for each search agent
            fitness=objf(Positions[i,:])
            
            # Update Alpha, Beta, and Delta
            if fitness<Alpha_score :
                Alpha_score=fitness; # Update alpha
                Alpha_pos=Positions[i,:].copy()
            
            
            if (fitness>Alpha_score and fitness<Beta_score ):
                Beta_score=fitness  # Update beta
                Beta_pos=Positions[i,:].copy()
            
            
            if (fitness>Alpha_score and fitness>Beta_score and fitness<Delta_score): 
                Delta_score=fitness # Update delta
                Delta_pos=Positions[i,:].copy()
            
        
        
        z=numpy.random.uniform()
        z=4*z*(1-z)
        
        a=2-l*((2)/Max_iter)+z; # a decreases linearly fron 2 to 0
        
        beta=0.5
        sigma=(math.gamma(1+beta)*math.sin(math.pi*beta/2)/(math.gamma((1+beta)/2)*beta*2**((beta-1)/2)))**(1/beta);
        
        # Update the Position of search agents including omegas
        for i in range(0,SearchAgents_no):
            for j in range (0,dim):     
                           
                r1=random.random() # r1 is a random number in [0,1]
                r2=random.random() # r2 is a random number in [0,1]
                
                A1=2*a*r1-a; # Equation (3.3)
                C1=2*r2; # Equation (3.4)
                
                D_alpha=abs(C1*Alpha_pos[j]-Positions[i,j]); # Equation (3.5)-part 1
                X1=Alpha_pos[j]-A1*D_alpha; # Equation (3.6)-part 1
                #else:
                 #   X1=random.random()
                #X1=numpy.random.uniform(0,1,(SearchAgents_no,dim)) *(ub-lb)+lb
                           
                r1=random.random()
                r2=random.random()
                
                A2=2*a*r1-a; # Equation (3.3)
                C2=2*r2; # Equation (3.4)
                
                D_beta=abs(C2*Beta_pos[j]-Positions[i,j]); # Equation (3.5)-part 2
                #if j%2==1:
                X2=Beta_pos[j]-A2*D_beta; # Equation (3.6)-part 2
                #else:
                #    X2=random.random()
                
                r1=random.random()
                r2=random.random() 
                
                A3=2*a*r1-a; # Equation (3.3)
                C3=2*r2; # Equation (3.4)
                
                D_delta=abs(C3*Delta_pos[j]-Positions[i,j]); # Equation (3.5)-part 3
                X3=Delta_pos[j]-A3*D_delta; # Equation (3.5)-part 3             
                
                Positions[i,j]=(X1+X2+X3)/3  # Equation (3.7)
                #rr=levy(,j)
                
            
            #ggnn=alpha(i,Max_iter)
            s=Positions[i,:]
            lamb=0.3
            
            u=numpy.random.uniform(size=len(s))*sigma
            v=numpy.random.uniform(size=len(s))
            step=u*abs(v)**(1/beta)
            stepsize=0.1*step*(s-Alpha_pos) #s-alpha_pos
            s=s+stepsize*numpy.random.uniform(size=len(s))
            Positions[i]=simplebounds(s,lb,ub)
            rand=random.random()
            if(Positions[i-1,j]>=lamb):
                positions[i,j]=1

            else:
                positions[i,j]=0
        Convergence_curve[l]=Alpha_score
        """if Convergence_curve[l]==Convergence_curve[l-1]==Convergence_curve[l-2]==Convergence_curve[l-3]==Convergence_curve[l-4]==Convergence_curve[l-5]==Convergence_curve[l-6]:
            print(Convergence_curve[l],Convergence_curve[l-6])
            break"""

        if (l%1==0):
               print(['At iteration '+ str(l)+ ' the best fitness is '+ str(Alpha_score)]);
    
    timerEnd=time.time()  
    sol.endTime=time.strftime("%Y-%m-%d-%H-%M-%S")
    sol.executionTime=timerEnd-timerStart
    sol.convergence=Convergence_curve
    sol.optimizer="GWO"
    sol.objfname=objf.__name__
    sol.no_of_iterations=l
    sol.positions=positions
    
    
    return sol
  
  

In [118]:
gg=GWO2(objf=F1,Max_iter=1000,lb=-100,ub=100,dim=30,SearchAgents_no=5)


['At iteration 0 the best fitness is 77536.06103547299']
['At iteration 1 the best fitness is 77536.06103547299']
['At iteration 2 the best fitness is 77536.06103547299']
['At iteration 3 the best fitness is 74528.19431102589']
['At iteration 4 the best fitness is 55376.05144452295']
['At iteration 5 the best fitness is 55376.05144452295']
['At iteration 6 the best fitness is 54516.74226948464']
['At iteration 7 the best fitness is 54516.74226948464']
['At iteration 8 the best fitness is 45351.48770896678']
['At iteration 9 the best fitness is 45351.48770896678']
['At iteration 10 the best fitness is 45351.48770896678']
['At iteration 11 the best fitness is 45351.48770896678']
['At iteration 12 the best fitness is 45351.48770896678']
['At iteration 13 the best fitness is 37887.90567573885']
['At iteration 14 the best fitness is 37887.90567573885']
['At iteration 15 the best fitness is 37887.90567573885']
['At iteration 16 the best fitness is 37887.90567573885']
['At iteration 17 the be

['At iteration 217 the best fitness is 3.442000261106038']
['At iteration 218 the best fitness is 3.442000261106038']
['At iteration 219 the best fitness is 3.442000261106038']
['At iteration 220 the best fitness is 2.7495825727139325']
['At iteration 221 the best fitness is 2.7495825727139325']
['At iteration 222 the best fitness is 2.7495825727139325']
['At iteration 223 the best fitness is 2.7495825727139325']
['At iteration 224 the best fitness is 2.7495825727139325']
['At iteration 225 the best fitness is 2.7495825727139325']
['At iteration 226 the best fitness is 2.2910280434311505']
['At iteration 227 the best fitness is 1.7838443160709003']
['At iteration 228 the best fitness is 1.7838443160709003']
['At iteration 229 the best fitness is 1.7838443160709003']
['At iteration 230 the best fitness is 1.6247465964137227']
['At iteration 231 the best fitness is 1.6247465964137227']
['At iteration 232 the best fitness is 1.6247465964137227']
['At iteration 233 the best fitness is 0.97

['At iteration 459 the best fitness is 1.1853658695646053e-11']
['At iteration 460 the best fitness is 7.635116742476123e-12']
['At iteration 461 the best fitness is 7.635116742476123e-12']
['At iteration 462 the best fitness is 5.804338299807423e-12']
['At iteration 463 the best fitness is 5.804338299807423e-12']
['At iteration 464 the best fitness is 5.001948952354657e-12']
['At iteration 465 the best fitness is 4.137731020795803e-12']
['At iteration 466 the best fitness is 4.137731020795803e-12']
['At iteration 467 the best fitness is 4.137731020795803e-12']
['At iteration 468 the best fitness is 3.5263560622287123e-12']
['At iteration 469 the best fitness is 3.3635019340319696e-12']
['At iteration 470 the best fitness is 2.7114112958805444e-12']
['At iteration 471 the best fitness is 2.2244564095190692e-12']
['At iteration 472 the best fitness is 2.2244564095190692e-12']
['At iteration 473 the best fitness is 1.55079778468706e-12']
['At iteration 474 the best fitness is 1.550797784

['At iteration 690 the best fitness is 4.1192390223197066e-22']
['At iteration 691 the best fitness is 4.1192390223197066e-22']
['At iteration 692 the best fitness is 4.1192390223197066e-22']
['At iteration 693 the best fitness is 4.1192390223197066e-22']
['At iteration 694 the best fitness is 3.685033650301802e-22']
['At iteration 695 the best fitness is 3.572926344349434e-22']
['At iteration 696 the best fitness is 3.111145306167482e-22']
['At iteration 697 the best fitness is 2.7156055401475788e-22']
['At iteration 698 the best fitness is 2.7156055401475788e-22']
['At iteration 699 the best fitness is 2.291955908954649e-22']
['At iteration 700 the best fitness is 1.6740331916315467e-22']
['At iteration 701 the best fitness is 1.4429980921527608e-22']
['At iteration 702 the best fitness is 1.4429980921527608e-22']
['At iteration 703 the best fitness is 1.4429980921527608e-22']
['At iteration 704 the best fitness is 1.4429980921527608e-22']
['At iteration 705 the best fitness is 1.148

['At iteration 902 the best fitness is 4.043698826920406e-28']
['At iteration 903 the best fitness is 3.844536891632796e-28']
['At iteration 904 the best fitness is 3.5613963145331966e-28']
['At iteration 905 the best fitness is 3.4931522210529497e-28']
['At iteration 906 the best fitness is 3.1478885297648082e-28']
['At iteration 907 the best fitness is 3.0521883720883316e-28']
['At iteration 908 the best fitness is 3.0521883720883316e-28']
['At iteration 909 the best fitness is 3.0521883720883316e-28']
['At iteration 910 the best fitness is 2.915662680190087e-28']
['At iteration 911 the best fitness is 2.807691940281027e-28']
['At iteration 912 the best fitness is 2.542236869233028e-28']
['At iteration 913 the best fitness is 2.542236869233028e-28']
['At iteration 914 the best fitness is 2.4898200250304153e-28']
['At iteration 915 the best fitness is 2.4608000794101563e-28']
['At iteration 916 the best fitness is 2.2997284188723405e-28']
['At iteration 917 the best fitness is 2.29972

In [120]:
gg.positions[0]

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 1, 1, 0])

In [4]:
import math

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [5]:
def F1(x):
    s=numpy.sum(x**2);
    return s

In [6]:
import numpy as np
import pandas as pd

In [110]:
liwc=pd.read_csv("LIWC2015 Results (twitter-sanders-apple3).csv")

In [111]:
liwc["Source (A)"].unique()

array(['class', 'Pos', 'Neg', 'Neutral'], dtype=object)

In [121]:
ynew=liwc["Source (A)"].values

In [122]:
liwc.drop("Source (B)",axis=1,inplace=True)

In [123]:
binary(liwc,0)

SVM
Selected 0.625
All 0.6048387096774193
KNN
Selected 0.625
All 0.6048387096774193


In [197]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.simplefilter('ignore')

clf=SVC(kernel='poly',degree=2,C=0.01)
clf6=KNeighborsClassifier(n_neighbors=5)
clf2=LogisticRegression(C=2)


def findPositionArray2(dimension):
    egwo=GWO(objf=F1,Max_iter=1000,lb=-100,ub=100,dim=dimension,SearchAgents_no=5)
    egwo.no_of_iterations,egwo.convergence[len(egwo.convergence)-1]
    newPos=egwo.positions[0]
    newnewPos=[]
    for i in range(len(egwo.positions[0])):
        if newPos[i]==0:
            newnewPos.append(False)
        else:
            newnewPos.append(True)
    return newnewPos 

def findPositionArray(dimension):
    egwo=GWO(objf=F1,Max_iter=1000,lb=-100,ub=100,dim=dimension,SearchAgents_no=5)
    egwo.no_of_iterations,egwo.convergence[len(egwo.convergence)-1]
    newPos=egwo.positions[0]
    newnewPos=[]
    for i in range(len(egwo.positions[0])):
        if newPos[i]==0:
            newnewPos.append(False)
        else:
            newnewPos.append(True)
    return newnewPos

def count(newnewPos):
    cou=0
    for i in newnewPos:
        if i==1:
            cou+=1
    return cou

def applyBinary(data,newnewPos):
    datanew=data.loc[:,newnewPos]
    return datanew

def SvmAl(clf,x_train_all,x_test_all,y_train_all,y_test_all):
    clf.fit(x_train_all,y_train_all)
    ypredsvm=clf.predict(x_test_all)
    #print("all     :",clf.score(x_train_all,y_train_all),clf.score(x_test_all,y_test_all))
    return clf.score(x_test_all,y_test_all)
    
def SvmSel(clf,x_train,x_test,y_train,y_test):
    clf.fit(x_train,y_train)
    ypredsvm=clf.predict(x_test)
    #print("selected:",clf.score(x_train,y_train),clf.score(x_test,y_test))
    return clf.score(x_test,y_test)
    
def knnSel(clf6,x_train,x_test,y_train,y_test):
    clf6.fit(x_train,y_train)
    ypredKNN=clf6.predict(x_test)
    #print("selected:",clf6.score(x_train,y_train),clf6.score(x_test,y_test))
    return clf6.score(x_test,y_test)

def knnAl(clf6,x_train_all,x_test_all,y_train_all,y_test_all):
    clf6.fit(x_train_all,y_train_all)
    ypredKNN=clf6.predict(x_test_all)
    #print("all     :",clf6.score(x_train_all,y_train_all),clf6.score(x_test_all,y_test_all))
    return clf6.score(x_test_all,y_test_all)

def function(dataset,name,clf=clf,clf6=clf6):
    svmSelected=[]
    svmAll=[]
    knnSelected=[]
    knnAll=[]
    #from sklearn.datasets import load_iris
    #iris=load_iris()
    allData=pd.DataFrame(dataset.data)

    newnewPos=findPositionArray(len(dataset.data[0]))
    newnewPos2=findPositionArray2(len(dataset.data[0]))
    allnew=applyBinary(allData,newnewPos)
    allnew2=applyBinary(allData,newnewPos2)
    noOfFeatures=count(newnewPos)
    noOfFeatures2=count(newnewPos2)
    final=[]
    final.append(name)

    x_train,x_test,y_train,y_test=train_test_split(allnew.values,dataset.target,random_state=0)
    x_train_all,x_test_all,y_train_all,y_test_all=train_test_split(dataset.data,dataset.target,random_state=0)
    
    print("Bgwo1")
    print("total no of features=",len(newnewPos))
    final.append(len(newnewPos))
    final.append(noOfFeatures)
    print("no of selected features=",noOfFeatures)
    print("SVM")
    for i in range(10):
        svmSelected.append(SvmSel(clf,x_train,x_test,y_train,y_test))
    print("Selected",sum(svmSelected)/10)
    final.append(sum(svmSelected)/10)
    
    for i in range(10):
        svmAll.append(SvmAl(clf,x_train_all,x_test_all,y_train_all,y_test_all))
    print("All",sum(svmAll)/10)
    final.append(sum(svmAll)/10)
    
    print("KNN")
    for i in range(10):
        knnSelected.append(knnSel(clf6,x_train,x_test,y_train,y_test))
    print("Selected",sum(svmSelected)/10)
    final.append(sum(svmSelected)/10)
    
    for i in range(10):
        knnAll.append(knnAl(clf6,x_train_all,x_test_all,y_train_all,y_test_all))
    print("All",sum(svmAll)/10)
    final.append(sum(svmAll)/10)
    
    
    x_train,x_test,y_train,y_test=train_test_split(allnew2.values,dataset.target,random_state=0)
    x_train_all,x_test_all,y_train_all,y_test_all=train_test_split(dataset.data,dataset.target,random_state=0)
    
    svmSelected=[]
    svmAll=[]
    knnSelected=[]
    knnAll=[]
    
    print()
    print("Bgow2")
    print("total no of features=",len(newnewPos2))
    print("no of selected features=",noOfFeatures2)
    print("SVM")
    for i in range(10):
        svmSelected.append(SvmSel(clf,x_train,x_test,y_train,y_test))
    print("Selected",sum(svmSelected)/10)
    
    for i in range(10):
        svmAll.append(SvmAl(clf,x_train_all,x_test_all,y_train_all,y_test_all))
    print("All",sum(svmAll)/10)
    
    print("KNN")
    for i in range(10):
        knnSelected.append(knnSel(clf6,x_train,x_test,y_train,y_test))
    print("Selected",sum(svmSelected)/10)
    
    for i in range(10):
        knnAll.append(knnAl(clf6,x_train_all,x_test_all,y_train_all,y_test_all))
    print("All",sum(svmAll)/10)
    
    final.append(sum(svmSelected)/10)
    final.append(sum(svmAll)/10)
    final.append(sum(svmSelected)/10)
    final.append(sum(svmAll)/10)
    
    return final
    


def binary(ion,name,no,clf=clf,clf6=clf6):
    output=ion[ion.columns[no]].values
    del ion[ion.columns[no]]
    newnewPos=findPositionArray(len(ion.columns))
    newnewPos2=findPositionArray2(len(ion.columns))
    allnew=applyBinary(ion,newnewPos)
    allnew2=applyBinary(ion,newnewPos2)
    noOfFeatures=count(newnewPos)
    noOfFeatures2=count(newnewPos2)
    
    final=[]
    final.append(name)
    final.append(len(newnewPos))
    final.append(noOfFeatures)

    svmSelected=[]
    svmAll=[]
    knnSelected=[]
    knnAll=[]
    
    x_train,x_test,y_train,y_test=train_test_split(allnew.values,output,random_state=0)
    x_train_all,x_test_all,y_train_all,y_test_all=train_test_split(ion.values,output,random_state=0)

    print("Bgwo1")
    print("total no of features=",len(newnewPos))
    print("no of selected features=",noOfFeatures)
    print("SVM")
    for i in range(10):
        svmSelected.append(SvmSel(clf,x_train,x_test,y_train,y_test))
    print("Selected",sum(svmSelected)/10)
    
    for i in range(10):
        svmAll.append(SvmAl(clf,x_train_all,x_test_all,y_train_all,y_test_all))
    print("All",sum(svmAll)/10)
    
    print("KNN")
    for i in range(10):
        knnSelected.append(knnSel(clf6,x_train,x_test,y_train,y_test))
    print("Selected",sum(svmSelected)/10)
    
    for i in range(10):
        knnAll.append(knnAl(clf6,x_train_all,x_test_all,y_train_all,y_test_all))
    print("All",sum(svmAll)/10)
    final.append(sum(svmSelected)/10)
    final.append(sum(svmAll)/10)
    final.append(sum(svmSelected)/10)
    final.append(sum(svmAll)/10)
    
    
    svmSelected=[]
    svmAll=[]
    knnSelected=[]
    knnAll=[]
    
    x_train,x_test,y_train,y_test=train_test_split(allnew2.values,output,random_state=0)
    x_train_all,x_test_all,y_train_all,y_test_all=train_test_split(ion.values,output,random_state=0)
    
    print()
    print("Bgow2")
    print("total no of features=",len(newnewPos2))
    print("no of selected features=",noOfFeatures2)
    print("SVM")
    for i in range(10):
        svmSelected.append(SvmSel(clf,x_train,x_test,y_train,y_test))
    print("Selected",sum(svmSelected)/10)
    
    for i in range(10):
        svmAll.append(SvmAl(clf,x_train_all,x_test_all,y_train_all,y_test_all))
    print("All",sum(svmAll)/10)
    
    print("KNN")
    for i in range(10):
        knnSelected.append(knnSel(clf6,x_train,x_test,y_train,y_test))
    print("Selected",sum(svmSelected)/10)
    
    for i in range(10):
        knnAll.append(knnAl(clf6,x_train_all,x_test_all,y_train_all,y_test_all))
    print("All",sum(svmAll)/10)
    final.append(sum(svmSelected)/10)
    final.append(sum(svmAll)/10)
    final.append(sum(svmSelected)/10)
    final.append(sum(svmAll)/10)
    
    return final

In [231]:
finalfinal=[]

In [233]:

from sklearn.datasets import load_breast_cancer,load_iris,load_wine,load_digits
breast=load_breast_cancer()
iris=load_iris()
wine=load_wine()
digits=load_digits()
finalfinal.append(function(breast,"breast",clf6,clf6))
finalfinal.append(function(iris,"iris",clf6,clf6))
finalfinal.append(function(wine,"wine",clf6,clf6))
finalfinal.append(function(digits,"digits",clf6,clf6))

Bgwo1
total no of features= 30
no of selected features= 16
SVM
Selected 0.916083916083916
All 0.9370629370629369
KNN
Selected 0.916083916083916
All 0.9370629370629369

Bgow2
total no of features= 30
no of selected features= 12
SVM
Selected 0.951048951048951
All 0.9370629370629369
KNN
Selected 0.951048951048951
All 0.9370629370629369
Bgwo1
total no of features= 4
no of selected features= 2
SVM
Selected 0.9736842105263157
All 0.9736842105263157
KNN
Selected 0.9736842105263157
All 0.9736842105263157

Bgow2
total no of features= 4
no of selected features= 3
SVM
Selected 0.9210526315789471
All 0.9736842105263157
KNN
Selected 0.9210526315789471
All 0.9736842105263157
Bgwo1
total no of features= 13
no of selected features= 5
SVM
Selected 0.8222222222222222
All 0.7333333333333333
KNN
Selected 0.8222222222222222
All 0.7333333333333333

Bgow2
total no of features= 13
no of selected features= 5
SVM
Selected 0.8444444444444443
All 0.7333333333333333
KNN
Selected 0.8444444444444443
All 0.7333333333

In [183]:
finalfinal

[['breast',
  30,
  13,
  0.9370629370629369,
  0.9370629370629369,
  0.9370629370629369,
  0.9370629370629369,
  0.9300699300699302,
  0.9370629370629369,
  0.9300699300699302,
  0.9370629370629369],
 ['iris',
  4,
  1,
  0.605263157894737,
  0.9736842105263157,
  0.605263157894737,
  0.9736842105263157,
  0.9473684210526316,
  0.9736842105263157,
  0.9473684210526316,
  0.9736842105263157],
 ['wine',
  13,
  6,
  0.7111111111111111,
  0.7333333333333333,
  0.7111111111111111,
  0.7333333333333333,
  0.7333333333333333,
  0.7333333333333333,
  0.7333333333333333,
  0.7333333333333333],
 ['digits',
  64,
  36,
  0.9622222222222222,
  0.9800000000000002,
  0.9622222222222222,
  0.9800000000000002,
  0.9777777777777776,
  0.9800000000000002,
  0.9777777777777776,
  0.9800000000000002]]

In [236]:
ion=pd.read_csv("ionosphere.data.csv")
finalfinal.append(binary(ion,"ion",34))

Bgwo1
total no of features= 34
no of selected features= 15
SVM
Selected 0.6363636363636365
All 0.6363636363636365
KNN
Selected 0.6363636363636365
All 0.6363636363636365

Bgow2
total no of features= 34
no of selected features= 16
SVM
Selected 0.6363636363636365
All 0.6363636363636365
KNN
Selected 0.6363636363636365
All 0.6363636363636365


In [240]:
sonar=pd.read_csv("sonar.all-data.csv")
finalfinal.append(binary(sonar,"sonar",60))

Bgwo1
total no of features= 60
no of selected features= 26
SVM
Selected 0.5192307692307693
All 0.5192307692307693
KNN
Selected 0.5192307692307693
All 0.5192307692307693

Bgow2
total no of features= 60
no of selected features= 30
SVM
Selected 0.5192307692307693
All 0.5192307692307693
KNN
Selected 0.5192307692307693
All 0.5192307692307693


In [241]:
def f(x):
    if x=='x':
        return 1
    if x=='o':
        return -1
    if x=='b':
        return 0

In [244]:
tictactoe=pd.read_csv("tic-tac-toe.data.csv.csv")
coll=tictactoe.columns[:-1]
for i in coll:
    tictactoe[i+"-2"]=tictactoe[i].apply(f)
for i in coll:
    del tictactoe[i]
finalfinal.append(binary(tictactoe,"ticTacToe",0))

Bgwo1
total no of features= 9
no of selected features= 6
SVM
Selected 0.6416666666666667
All 0.6416666666666667
KNN
Selected 0.6416666666666667
All 0.6416666666666667

Bgow2
total no of features= 9
no of selected features= 2
SVM
Selected 0.6416666666666667
All 0.6416666666666667
KNN
Selected 0.6416666666666667
All 0.6416666666666667


In [248]:
zoo=pd.read_csv("zoo.data.csv")
del zoo["aardvark"]
finalfinal.append(binary(zoo,"zoo",16))

Bgwo1
total no of features= 16
no of selected features= 11
SVM
Selected 0.5600000000000002
All 0.5600000000000002
KNN
Selected 0.5600000000000002
All 0.5600000000000002

Bgow2
total no of features= 16
no of selected features= 9
SVM
Selected 0.5600000000000002
All 0.5600000000000002
KNN
Selected 0.5600000000000002
All 0.5600000000000002


In [252]:
lymph=pd.read_csv("lymphography.data.csv")
finalfinal.append(binary(lymph,"lymphography",0))

Bgwo1
total no of features= 18
no of selected features= 9
SVM
Selected 0.6756756756756755
All 0.6486486486486488
KNN
Selected 0.6756756756756755
All 0.6486486486486488

Bgow2
total no of features= 18
no of selected features= 12
SVM
Selected 0.6756756756756755
All 0.6486486486486488
KNN
Selected 0.6756756756756755
All 0.6486486486486488


In [270]:
heart=pd.read_csv("heart.dat.csv",delimiter=" ")
heart.shape
finalfinal.append(binary(heart,"heart",13))

Bgwo1
total no of features= 13
no of selected features= 7
SVM
Selected 0.7647058823529411
All 0.7647058823529411
KNN
Selected 0.7647058823529411
All 0.7647058823529411

Bgow2
total no of features= 13
no of selected features= 4
SVM
Selected 0.7352941176470589
All 0.7647058823529411
KNN
Selected 0.7352941176470589
All 0.7647058823529411


In [279]:
def g(x):
    if x=='y':
        return 1
    if x=='n':
        return 0

In [287]:
congress=pd.read_csv("house-votes-84.data.csv")
congress
coll=congress.columns[1:]
for i in coll:
    congress[i+"-2"]=congress[i].apply(g)
for i in coll:
    del congress[i]
congress.dropna(inplace=True)
finalfinal.append(binary(congress,"congress",0))

Bgwo1
total no of features= 16
no of selected features= 9
SVM
Selected 0.48275862068965514
All 0.48275862068965514
KNN
Selected 0.48275862068965514
All 0.48275862068965514

Bgow2
total no of features= 16
no of selected features= 10
SVM
Selected 0.48275862068965514
All 0.48275862068965514
KNN
Selected 0.48275862068965514
All 0.48275862068965514


In [288]:
table=pd.DataFrame(finalfinal)

In [289]:
table.columns=["dataset","total features","Selected features","SVM on selected(bGwo1)","svm on all","Knn on selected","knn on all",
              "svm on selected(bGwo2)","svm on all","knn on selected","knn on all"]

In [290]:

table

,dataset,total features,Selected features,SVM on selected(bGwo1),svm on all,Knn on selected,knn on all,svm on selected(bGwo2),svm on all,knn on selected,knn on all
0,breast,30,18,0.944056,0.937063,0.944056,0.937063,0.944056,0.937063,0.944056,0.937063
1,iris,4,2,0.973684,0.973684,0.973684,0.973684,0.605263,0.973684,0.605263,0.973684
2,wine,13,9,0.733333,0.733333,0.733333,0.733333,0.800000,0.733333,0.800000,0.733333
3,digits,64,33,0.940000,0.980000,0.940000,0.980000,0.971111,0.980000,0.971111,0.980000
4,breast,30,16,0.916084,0.937063,0.916084,0.937063,0.951049,0.937063,0.951049,0.937063
5,iris,4,2,0.973684,0.973684,0.973684,0.973684,0.921053,0.973684,0.921053,0.973684
6,wine,13,5,0.822222,0.733333,0.822222,0.733333,0.844444,0.733333,0.844444,0.733333
7,digits,64,32,0.933333,0.980000,0.933333,0.980000,0.975556,0.980000,0.975556,0.980000
8,ion,34,16,0.636364,0.636364,0.636364,0.636364,0.636364,0.636364,0.636364,0.636364
9,ion,34,23,0.636364,0.636364,0.636364,0.636364,0.636364,0.636364,0.636364,0.636364


In [292]:
table.to_csv("datasets.csv",sep=",")

In [277]:
df=pd.read_csv("datasets.csv")
df

,Unnamed: 0,dataset,total features,Selected features,SVM on selected(bGwo1),svm on all,Knn on selected,knn on all,svm on selected(bGwo2),svm on all.1,knn on selected,knn on all.1
0,0,breast,30,18,0.944056,0.937063,0.944056,0.937063,0.944056,0.937063,0.944056,0.937063
1,1,iris,4,2,0.973684,0.973684,0.973684,0.973684,0.605263,0.973684,0.605263,0.973684
2,2,wine,13,9,0.733333,0.733333,0.733333,0.733333,0.800000,0.733333,0.800000,0.733333
3,3,digits,64,33,0.940000,0.980000,0.940000,0.980000,0.971111,0.980000,0.971111,0.980000
4,4,breast,30,16,0.916084,0.937063,0.916084,0.937063,0.951049,0.937063,0.951049,0.937063
5,5,iris,4,2,0.973684,0.973684,0.973684,0.973684,0.921053,0.973684,0.921053,0.973684
6,6,wine,13,5,0.822222,0.733333,0.822222,0.733333,0.844444,0.733333,0.844444,0.733333
7,7,digits,64,32,0.933333,0.980000,0.933333,0.980000,0.975556,0.980000,0.975556,0.980000
8,8,ion,34,16,0.636364,0.636364,0.636364,0.636364,0.636364,0.636364,0.636364,0.636364
9,9,ion,34,23,0.636364,0.636364,0.636364,0.636364,0.636364,0.636364,0.636364,0.636364
